In [96]:
#HW implementation

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

t = [0.25, 0.5, 1, 2, 3, 5, 7, 10, 20]
zero_curve = [0.008, 0.0085, 0.0092, 0.01,  0.0106, 0.0119, 0.0132, 0.015, 0.02]

sig = 0.01
a = 0.1
fixed_rate = 0.0149

timesteps = np.arange(0, 10.25, 0.25)

#print(len(timesteps))

cs = CubicSpline(t, zero_curve)
spot_rate = cs(timesteps)
df = np.exp(- timesteps * spot_rate)

print(df)

dt = np.zeros(len(timesteps))
fwd = np.zeros(len(timesteps))
beta = np.zeros(len(timesteps))
B = np.zeros(len(timesteps))
fwdswapval = np.zeros(len(timesteps))

fwd[0] = spot_rate[0]
for i in range(1, len(timesteps)):
    dt[i] = timesteps[i] - timesteps[i - 1]
    fwd[i] = -np.log(df[i]/df[i - 1])/dt[i]
    beta[i] = fwd[i] + (sig * (1 - np.exp(-a *  timesteps[i]))/ (a * np.sqrt(2)))**2
    B[i] = (1 - np.exp(-a * timesteps[i])) / a
    

for i in range(1, len(timesteps) ):
    fwdswapval[i -1] = np.sum(df[i:len(timesteps)] * dt[i - 1] * fixed_rate) - (df[i - 1] - df[len(timesteps) - 1])
    
df_calib = np.zeros((len(timesteps), len(timesteps)))

for i in range(len(timesteps)):
    for j in range(i, len(timesteps)):
        df_calib[i, j] = df[j] / df[i] * np.exp(B[j - i] * fwd[i] - (sig ** 2) * (1 - np.exp(-2 * a * timesteps[i])) * (B[j - i] ** 2) / 4 *a)
        
#print(df_calib)

param1 = np.exp(-a * 0.25)  # 0.25 should be dt[i] 
param2 = np.sqrt((1 - np.exp(-2 * a * 0.25))/(2 * a)) #again hard coded dt

#print(param1, param2)

short_rate = np.zeros(len(timesteps) - 1)

sims = 3

df_sim = np.zeros((sims, len(timesteps) - 1, len(timesteps) - 1))

short_rate[0] = spot_rate[0]
for k in range(len(timesteps) - 1):
    df_sim[:, 0, k] = df_calib[0, k] * np.exp(- B[k + 1] * short_rate[0])
    #print(B[k + 1])
    #print(short_rate[0])
    
df_sim[:, 0, 0] = 1

print("      ")

for i in range(sims):
    for j in range(1, len(timesteps) - 1):
        z1 = np.random.normal(0,1)
        short_rate[j] = short_rate[j - 1] * param1 + z1 * sig * param2 + beta[j] - beta[j - 1] * param1
        for k in range(len(timesteps) - 1):
            df_sim[i,  j,  k] = df_calib[j, k] * np.exp(- B[k + 1] * short_rate[j])
            df_sim[i, j, j] = 1
        

for i in range(1, len(timesteps) ):
    fwdswapval[i -1] = np.sum(df[i:len(timesteps)] * dt[i - 1] * fixed_rate) - (df[i - 1] - df[len(timesteps) - 1])


[1.         0.998002   0.99575902 0.99335397 0.99084219 0.9882548
 0.9856131  0.98292807 0.98019867 0.97741551 0.97457443 0.97167151
 0.9687003  0.96565365 0.96252988 0.95932939 0.95605327 0.95270322
 0.94928153 0.94579113 0.94223553 0.93861836 0.93494177 0.93120758
 0.92741766 0.92357395 0.91967845 0.91573322 0.91174038 0.907702
 0.90361973 0.89949512 0.89532966 0.89112485 0.88688214 0.88260295
 0.87828869 0.87394073 0.86956042 0.86514907 0.86070798]
      
[[[1.         0.99623387 0.99403838 ... 0.8733109  0.8689492  0.86455597]
  [0.         1.         0.99002634 ... 0.84534188 0.84069673 0.83603592]
  [0.         0.         1.         ... 0.87584506 0.87152204 0.8671656 ]
  ...
  [0.         0.         0.         ... 1.         1.04045908 1.04075962]
  [0.         0.         0.         ... 0.         1.         1.05691237]
  [0.         0.         0.         ... 0.         0.         1.        ]]

 [[1.         0.99623387 0.99403838 ... 0.8733109  0.8689492  0.86455597]
  [0.      